In [135]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import sqlite3
import numpy as np
import random
import urllib.request
import time
import os

In [149]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
result = []
rows = curs.execute('select Livsmedelsnamn, Livsmedelsnummer from livs limit 2000')
for row in rows:
        result.append(row)

#rows = np.array(result)
#rows = rows[970:]
samples = random.sample(range(0, 1999), 3)
sampledFoodItems = []
#print (type(sampledFoodItems))

for index in samples:
    sampledFoodItems.append(result[index])

sampledFoodItems = np.array(sampledFoodItems)
print (samples, sampledFoodItems)

conn.close()

[209, 939, 1774] [['Sjömansbiff fryst' '256']
 ['Köttfärssås älgfärs' '1132']
 ['Sjömansbiff' '2092']]


In [150]:
def download_page(url): #urllib library for Extracting web pages
    try:
        headers = {}
        headers['User-Agent'] = "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36"
        req = urllib.request.Request(url, headers = headers)
        resp = urllib.request.urlopen(req)
        respData = str(resp.read())
        return respData
    except Exception as e:
        print(str(e))

In [155]:
#Finding 'Next Image' from the given raw page
def _images_get_next_item(s):
    print (type(s))
    start_line = s.find('rg_di')
    if start_line == -1:    #If no links are found then give an error!
        end_quote = 0
        link = "no_links"
        return link, end_quote
    else:
        start_line = s.find('"class="rg_meta"')
        start_content = s.find('"ou"',start_line+1)
        end_content = s.find(',"ow"',start_content+1)
        content_raw = str(s[start_content+6:end_content-1])
        return content_raw, end_content

In [156]:
def _images_get_all_items(page):
    items = []
    while True:
        item, end_content = _images_get_next_item(page)
        if item == "no_links":
            break
        else:
            items.append(item)      #Append all the links in the list named 'Links'
            time.sleep(0.1)        #Timer could be used to slow down the request for image downloads
            page = page[end_content:]
    return items


In [157]:
t0 = time.time()   #start the timer
keywords = sampledFoodItems
#Prepare keywords
for i, item in enumerate(keywords):
    keywords[i][0]=keywords[i][0].replace(' ','%20')
print (keywords)

[['Sjömansbiff%20fryst' '256']
 ['Köttfärssås%20älgfä' '1132']
 ['Sjömansbiff' '2092']]


In [163]:
    #Download Image Links
    for index, item in enumerate(keywords):
        iteration = "Item no.: " + str(index) + " -->" + " Item name = " + str(item[0])
        print (iteration)
        print ("Evaluating...")
        print(item)

         #make a search keyword  directory
        try:
            os.makedirs(item[1])
        except OSError as e:
            if e.errno != 17:
                raise
            # time.sleep might help here
            pass


        url = 'https://www.google.se/search?q=' + item[0] +'&espv=2&biw=1366&bih=667&site=webhp&source=lnms&tbm=isch&sa=X&ei=XosDVaCXD8TasATItgE&ved=0CAcQ_AUoAg'
        print(url)
        #raw_html =  download_page(url)
        #print(raw_html)
        
        #time.sleep(0.1)
        #imageLinks = (_images_get_all_items(raw_html)) # + items

        print ("Image Links = "+str(imageLinks))
        print ("Total Image Links = "+str(len(imageLinks)))
        print ("\n")


Item no.: 0 --> Item name = Sjömansbiff%20fryst
Evaluating...
['Sjömansbiff%20fryst' '256']
https://www.google.se/search?q=Sjömansbiff%20fryst&espv=2&biw=1366&bih=667&site=webhp&source=lnms&tbm=isch&sa=X&ei=XosDVaCXD8TasATItgE&ved=0CAcQ_AUoAg
Image Links = ['https://static.mathem.se/shared/images/products/medium/tistel-olja-eko-500ml-urtekram-1.jpg', 'https://www.jordklok.se/images/2.8421/crearome-tistelolja-kallpressad-eko-100-ml.jpeg', 'https://grongava.se/images/db/item/778.jpg', 'https://static.mathem.se/shared/images/products/large/tistel-olja-eko-500ml-urtekram-1.jpg', 'http://www.svensktkosttillskott.se/images/tistelolja_eko_11247.jpg', 'http://ravaja.com/images/zoom/tistelolja-kallpressad-eko-crearome.jpg', 'https://static.mathem.se/shared/images/products/large/07330137040000_c1n1.jpg', 'http://www.shopping4net.se/Common/PCCs/Products/Grpx/H4Net/Img-HTKE0-98-500_1-large.jpg', 'https://shopcdn.textalk.se/shop/24672/art72/h6921/14236921-origpic-942d8c.jpg?max-width\\\\u003d240\\\

In [164]:
def downloadImages(keyword,number):
    k=0
    numberOfImages = 5 # Max is len(items)
    errorCount=0
    while(k<numberOfImages):
        import urllib.request
        import urllib.error

        try:
            req = urllib.request.Request(items[k], headers={"User-Agent": "Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.27 Safari/537.17"})
            response = urllib.request.urlopen(req,None,15)
            output_file = open(number+"/"+str(k+1)+".jpg",'wb')

            data = response.read()
            output_file.write(data)
            response.close();

            print("completed ====> "+str(k+1))

            k=k+1;

        except urllib.error.HTTPError as e:  #If there is any HTTPError

            errorCount+=1
            print("HTTPError"+str(k))
            k=k+1;
        except urllib.error.URLError as e:

            errorCount+=1
            print("URLError "+str(k))
            k=k+1;


In [165]:
#This allows you to write all the links into a test file. This text file will be created in the same directory as your code. You can comment out the below 3 lines to stop writing the output to the text file.
#info = open('output.txt', 'a')        #Open the text file called database.txt
#info.write(str(i) + ': ' + str(search_keyword[i-1]) + ": " + str(items) + "\n\n\n")         #Write the title of the page
#info.close()                            #Close the file

t1 = time.time()    #stop the timer
total_time = t1-t0   #Calculating the total time required to crawl, find and download all the links of 60,000 images
print("Total time taken: "+str(total_time)+" Seconds")
print ("Starting Download...")

## To save imges to the same directory
# IN this saving process we are just skipping the URL if there is any error
downloadImages(keyword)
i = i+1

print("\n")
print("Everything downloaded!")
print("\n"+str(errorCount)+" ----> total Errors")


Total time taken: 328.803640127182 Seconds
Starting Download...
completed ====> 1
completed ====> 2
completed ====> 3
completed ====> 4
completed ====> 5


Everything downloaded!

0 ----> total Errors
